In [2]:
import sys
import numpy as np
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import balanced_accuracy_score, make_scorer
from imblearn.over_sampling import RandomOverSampler
from sklearn import model_selection
from sklearn.svm import SVC
from sklearn.cluster import SpectralClustering
from sklearn.ensemble import RandomForestClassifier
from imblearn.ensemble import BalancedRandomForestClassifier
from imblearn.over_sampling import SMOTE
from sklearn.linear_model import LogisticRegression
#from xgboost import XGBClassifier
from sklearn.ensemble import BaggingClassifier

#Models
from sklearn.ensemble import RandomForestClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

sys.path.insert(0, '/Users/gracewang/Documents/GitHub/elecfinal')
sys.path.insert(0, 'D:\Fall23 Coursework\ELEC478\Competition\elecfinal')
from ml_pipeline import train_n_predict, validation, clean_split
from Data.data_cleaner import cleaner

In [3]:
## Clean data
train_path = "../Data/train_data.csv"
feature_path = "../Data/feature_weights.csv"
morph_path = "../Data/imputed_morph_embed.csv"
X_train, X_val, X_query, y_train, y_val, y_query = clean_split(train_path, feature_path, morph_path)

In [4]:
# Oversample X_train_feat
ros = RandomOverSampler(random_state=0, sampling_strategy = 'minority')
X_train, y_train = ros.fit_resample(
        X_train, y_train)

In [5]:
X_train_feat = X_train.copy()
X_val_feat = X_val.copy()
X_query_feat = X_query.copy()

In [6]:
def one_hot(column, df, suffix=''):
    """
    one-hot encodes this shit
    """
    cats = pd.unique(df[column])

    for cat in cats:
        new_col = cat+suffix
        df[new_col] = df[column]==cat
        df[new_col] = df[new_col].astype('int')
    
    df = df.drop(columns=column)
    return df

In [7]:
# one-hot encode brain areas for all
X_train_feat = one_hot('pre_brain_area', X_train_feat, '_pre')
X_train_feat = one_hot('post_brain_area', X_train_feat, '_post')

X_val_feat = one_hot('pre_brain_area', X_val_feat, '_pre')
X_val_feat = one_hot('post_brain_area', X_val_feat, '_post')

X_query_feat = one_hot('pre_brain_area', X_query_feat, '_pre')
X_query_feat = one_hot('post_brain_area', X_query_feat, '_post')

In [8]:
# encode brain areas
area1 = ["basal", "soma"]
area2 = ["axon", "apical", "oblique", "apical_shaft"]
area3 = ["apical_tuft"]

def area_cols(df):
    df["area1"] = df["compartment"].isin(area1).astype('int')
    df["area2"] = df["compartment"].isin(area2).astype('int')
    df["area3"] = df["compartment"].isin(area3).astype('int')
    df.drop(columns='compartment')
    return df

In [9]:
X_train_feat = area_cols(X_train_feat)
X_val_feat = area_cols(X_val_feat)
X_query_feat = area_cols(X_query_feat)

In [10]:
X_train_feat = X_train_feat.select_dtypes('number')
X_val_feat = X_val_feat.select_dtypes('number')
X_query_feat = X_query_feat.select_dtypes('number')

In [11]:
X_train_feat.dtypes

ID                                  int64
axonal_coor_x                     float64
axonal_coor_y                     float64
axonal_coor_z                     float64
dendritic_coor_x                  float64
dendritic_coor_y                  float64
dendritic_coor_z                  float64
adp_dist                          float64
post_skeletal_distance_to_soma    float64
pre_skeletal_distance_to_soma     float64
pre_oracle                        float64
pre_test_score                    float64
pre_rf_x                          float64
pre_rf_y                          float64
post_oracle                       float64
post_test_score                   float64
post_rf_x                         float64
post_rf_y                         float64
pre_nucleus_x                     float64
pre_nucleus_y                     float64
pre_nucleus_z                     float64
post_nucleus_x                    float64
post_nucleus_y                    float64
post_nucleus_z                    

Making submission

In [12]:
leaderboard_path = "../Data/leaderboard_data.csv"
sub_data = cleaner(leaderboard_path, feature_path, morph_path, submission = True)
sub_data = area_cols(sub_data)
sub_data = one_hot('pre_brain_area', sub_data, '_pre')
sub_data = one_hot('post_brain_area', sub_data, '_post')
sub_data = sub_data.select_dtypes('number')

In [13]:
sub_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42593 entries, 0 to 42592
Data columns (total 38 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   ID                              42593 non-null  int64  
 1   axonal_coor_x                   42593 non-null  float64
 2   axonal_coor_y                   42593 non-null  float64
 3   axonal_coor_z                   42593 non-null  float64
 4   dendritic_coor_x                42593 non-null  float64
 5   dendritic_coor_y                42593 non-null  float64
 6   dendritic_coor_z                42593 non-null  float64
 7   adp_dist                        42593 non-null  float64
 8   post_skeletal_distance_to_soma  42593 non-null  float64
 9   pre_skeletal_distance_to_soma   42593 non-null  float64
 10  pre_oracle                      42593 non-null  float64
 11  pre_test_score                  42593 non-null  float64
 12  pre_rf_x                        

In [18]:
X_val_feat.drop("ID", axis = 1).info()

<class 'pandas.core.frame.DataFrame'>
Index: 42587 entries, 4418 to 174173
Data columns (total 37 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   axonal_coor_x                   42587 non-null  float64
 1   axonal_coor_y                   42587 non-null  float64
 2   axonal_coor_z                   42587 non-null  float64
 3   dendritic_coor_x                42587 non-null  float64
 4   dendritic_coor_y                42587 non-null  float64
 5   dendritic_coor_z                42587 non-null  float64
 6   adp_dist                        42587 non-null  float64
 7   post_skeletal_distance_to_soma  42587 non-null  float64
 8   pre_skeletal_distance_to_soma   42587 non-null  float64
 9   pre_oracle                      42587 non-null  float64
 10  pre_test_score                  42587 non-null  float64
 11  pre_rf_x                        42587 non-null  float64
 12  pre_rf_y                        4

In [20]:
order = X_train_feat.drop("ID", axis = 1).columns

In [38]:
max_depth_list = np.linspace(1,10, num = 10, dtype=int)
num_features = np.linspace(1,20, num = 20, dtype = int)

In [39]:
max_depth_list

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10])

In [40]:
accuracies = {}
for depth in max_depth_list:
    for num in num_features:
        RF = RandomForestClassifier(max_depth=depth, max_features = num) 
        RF.fit(X_train_feat.drop("ID", axis = 1),y_train)
        y_hat_valid = RF.predict(X_val_feat[order])
        valid_acc = balanced_accuracy_score(y_val, y_hat_valid)
        accuracies[(depth, num)] = valid_acc
        print(f"depth: {depth}, num features {num}, valid accuracy, {valid_acc}")

depth: 1, num features 1, valid accuracy, 0.5680541272317969
depth: 1, num features 2, valid accuracy, 0.6035004439295912
depth: 1, num features 3, valid accuracy, 0.6735277922251905
depth: 1, num features 4, valid accuracy, 0.6992773092219149
depth: 1, num features 5, valid accuracy, 0.7074034863577022
depth: 1, num features 6, valid accuracy, 0.7334429777037019
depth: 1, num features 7, valid accuracy, 0.7186060594419096
depth: 1, num features 8, valid accuracy, 0.73064453027521
depth: 1, num features 9, valid accuracy, 0.7455393613192852
depth: 1, num features 10, valid accuracy, 0.7477643449674407
depth: 1, num features 11, valid accuracy, 0.7439672201339145
depth: 1, num features 12, valid accuracy, 0.750064808138359
depth: 1, num features 13, valid accuracy, 0.7204023407435427
depth: 1, num features 14, valid accuracy, 0.7490127738113064
depth: 1, num features 15, valid accuracy, 0.7332277030452519
depth: 1, num features 16, valid accuracy, 0.722837386489305
depth: 1, num feature

KeyboardInterrupt: 

In [43]:
best_params= max(accuracies, key=accuracies.get)
print(best_params)

(2, 10)


In [44]:
RF = RandomForestClassifier(max_depth=best_params[0], max_features = best_params[1]) 
RF.fit(X_train_feat.drop("ID", axis = 1),y_train)

RandomForestClassifier(max_depth=2, max_features=10)

In [45]:
leaderboard_predictions = RF.predict(sub_data[order])
sub_data["connected"] = leaderboard_predictions

In [46]:
submission_data = sub_data.filter(['ID','connected'])
submission_data.to_csv('submission_data.csv',index=False)